In [1]:
from dataloader import Dataset
from utils import * 

data = Dataset("Stage_1_Data_ver._3/stage1/box")
data.load_data()

visibility = data.visibility
images     = data.images
indices    = np.logical_and(visibility[:,0], visibility[:,1])
pts1, pts2 = data.points[indices, 0], data.points[indices, 1]

In [5]:
import copy

count_graph = copy.deepcopy(data.count_graph)

In [2]:
images_to_add, appended_ids = data.computeSceneGraph()
images_to_add

[95,
 99,
 93,
 91,
 97,
 88,
 86,
 33,
 82,
 79,
 77,
 75,
 84,
 31,
 73,
 68,
 64,
 59,
 57,
 62,
 66,
 53,
 106,
 108,
 104,
 102,
 110,
 44,
 48,
 42,
 51,
 46,
 28,
 26,
 24,
 22,
 39,
 37,
 55,
 17,
 15,
 19,
 13,
 11,
 35,
 71,
 8,
 6,
 2,
 0]

In [6]:
images_to_add = []
appended_ids  = []

for iteration in range(data.N_images - 1):

    if iteration == 0:
        max_ind = np.array((count_graph == count_graph.max()).nonzero())

        if max_ind.shape[1] > 2:
            selected_id = np.argmax(np.abs(max_ind[1,:] - max_ind[0,:]))
        else:
            selected_id = 0

        id_one = max_ind[0,selected_id]
        id_two = max_ind[1,selected_id]
        images_to_add.append(data.images[id_one])
        images_to_add.append(data.images[id_two])
        appended_ids.append(id_one)
        appended_ids.append(id_two)
        count_graph[id_one, id_two] = 0
        count_graph[id_two, id_one] = 0
    else:

        sub_graph = count_graph[:,appended_ids]
        max_ind = np.array((sub_graph == sub_graph.max()).nonzero())

        if max_ind.shape[1] > 2:
            selected_id = np.argmax(np.abs(max_ind[1,:] - max_ind[0,:]))
        else:
            selected_id = 0
        
        id_one = max_ind[0, selected_id]
        id_two = appended_ids[max_ind[1,selected_id]]
        
        images_to_add.append(data.images[id_one])
        
        appended_ids.append(id_one)
        
        for ids in appended_ids[:-1]:

            count_graph[id_one, ids] = 0
            count_graph[ids, id_one] = 0





In [3]:
np.unique(np.array(images_to_add)).shape

(50,)

In [68]:
sub_graph = count_graph[:,appended_ids]
print(sub_graph.shape)
max_ind = np.array((sub_graph == sub_graph.max()).nonzero())

if max_ind.shape[1] > 2:
    selected_id = np.argmax(np.abs(max_ind[1,:] - max_ind[0,:]))
else:
    selected_id = 0

id_one = max_ind[0, selected_id]
id_two = data.images_index[images_to_add[max_ind[1,selected_id]]]
print(id_one, id_two)
images_to_add.append(data.images[id_one])
appended_ids.append(id_one)
count_graph[id_one, id_two] = 0
count_graph[id_two, id_one] = 0


(50, 2)
41 42


In [69]:
images_to_add

[95, 99, 93]

In [63]:
max_ind[1,0]

0

In [2]:
E12, best_criterion = estimate_essential_matrix(pts1, pts2, data.K)
rotations, translations = extract_cam_pose(E12, data.K)
rot, trans = decomposeEssentialMat(E12)


(3, 3)


In [3]:
best_criterion

39

In [4]:
points_3d = []
for i in range(4):
    P1 = computeProjectionMatrix(data.K, np.eye(3), np.zeros((3,)))
    P2 = computeProjectionMatrix(data.K, rotations[i], translations[i])
    points_3d.append(linearTriangulation(pts1, pts2, P1, P2))

In [5]:
X,R,C = cheiralityCondition(points_3d, rotations, translations)

In [6]:
X_reconstructed   = np.zeros((data.points.shape[0], 3))
reconstructed_ind = np.zeros((data.points.shape[0], 1))

In [83]:
camera_rotations    = []
camera_translations = []

In [8]:
X_reconstructed[indices] = X
reconstructed_ind[indices] = 1


In [84]:
camera_rotations.append(np.eye(3))
camera_rotations.append(R)
camera_translations.append(np.zeros((3,)))
camera_translations.append(C)


In [85]:
camera_rotations

[array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[ 0.97494987, -0.22187048, -0.01569227],
        [ 0.21870572,  0.9691071 , -0.11401419],
        [ 0.04050387,  0.10772613,  0.99335518]])]

In [88]:
camera_translations

[array([0., 0., 0.]), array([-0.00436593, -0.02938401, -0.01329672])]

In [10]:
for image_number in range(2, data.N_images):
    print(image_number)
    
    visibility_points = data.visibility[:, image_number][...,None]
    new_points        = data.points[:,image_number]
    indices = np.logical_and(reconstructed_ind, visibility_points)[:,0]

    X_new       = X_reconstructed[indices]
    corr_points = new_points[indices]
    break
    

2


In [11]:
X_new_homo = to_homogeneous(X_new)
corr_points_homo = to_homogeneous(corr_points)

In [77]:
def reprojection_error(x,X_homo, K,R,C):

    P = computeProjectionMatrix(K,R,C)

    proj    = np.matmul(P, X_homo.T).T
    proj    = proj / proj[:,2][...,None]
    proj_2d = proj[:,:2]
    
    error = ((x - proj_2d)**2).sum(axis = 1)
    return error.mean()

def DLT_PnP(points_2d, points_3d, K):

    points_2d_homogeneous = to_homogeneous(points_2d)
    points_3d_homogeneous = to_homogeneous(points_3d)
    K_inv = np.linalg.inv(K)
    points_2d_camera = apply_K_inv(K_inv, points_2d_homogeneous)

    N_points = points_2d_homogeneous.shape[0]
    if N_points == 6:
        num_iters = 1
    else:
        num_iters = 2000
    
    best_R = None
    best_C = None
    best_criterion = np.inf
    for i in range(num_iters):
        
        points = np.random.choice(N_points,6, replace= False)
        camera_2d     = points_2d_camera[points]
        homogenous_3d = points_3d_homogeneous[points]

        A = np.zeros((12, 12))
        for p in range(6):
            A[2*p] = np.concatenate([ np.zeros(4,), -homogenous_3d[p] , camera_2d[p,1]*homogenous_3d[p]])
            A[(2*p) + 1] = np.concatenate([homogenous_3d[p],np.zeros(4,) , -camera_2d[p,0]*homogenous_3d[p]])
            
        U,S,V = np.linalg.svd(A)
        P = V[-1].reshape((3, 4))
        R = P[:, :3]
        u, s, v = np.linalg.svd(R) # to enforce Orthonormality
        R = u @ v

        C = P[:, 3].reshape((-1,1))
        C = np.matmul(- np.linalg.inv(R),C)
        if np.linalg.det(R) < 0:
            R = -R
            C = -C
        
        error = reprojection_error(points_2d[points], homogenous_3d, K,R,C)

        if error < best_criterion:
            best_criterion = error
            best_C = C
            best_R = R

    return best_R, best_C
            
        

In [78]:
R,t = DLT_PnP(corr_points,X_new, data.K)

445302.69744325866
270425.39530188614
122238.09470511368
9971.192828254856


In [79]:
R

array([[ 0.97494987, -0.22187048, -0.01569227],
       [ 0.21870572,  0.9691071 , -0.11401419],
       [ 0.04050387,  0.10772613,  0.99335518]])

In [80]:
t

array([[-0.05300088],
       [ 0.12234486],
       [ 0.16096744]])